In [ ]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [ ]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
####################### NASA Mars News ##########################

In [ ]:
# Mars news URL of page to be scraped
news_url = 'https://mars.nasa.gov/news/'

browser.visit(news_url)

html = browser.html

news_soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Retrieve the latest news title and paragraph
news_title = news_soup.find_all('div', class_='content_title')[0].text
news_p = news_soup.find_all('div', class_='article_teaser_body')[0].text

print(news_title)
print("--------------------------------------------------------------------")
print(news_p)

In [ ]:
######################## JPL Mars Space Images - Featured Image ##########################

In [ ]:
# Mars Image to be scraped
jpl_nasa_url = 'https://www.jpl.nasa.gov'
images_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(images_url)

html = browser.html

images_soup = BeautifulSoup(html, 'html.parser')

In [ ]:
relative_image_path = images_soup.find_all('img')[3]["src"]
featured_image_url = jpl_nasa_url + relative_image_path
print(featured_image_url)

In [ ]:

######################## Mars Weather ##########################

In [ ]:
# Mars weather to be scraped
weather_url = 'https://twitter.com/marswxreport?lang=en'

browser.visit(weather_url)

weather_html = browser.html

weather_soup = BeautifulSoup(weather_html, 'html.parser')

In [ ]:
# Retrieve latest tweet with Mars weather info
mars_weather = weather_soup.find_all('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')[0].text
mars_weather

In [ ]:
######################## Mars Facts ##########################

In [ ]:
# Mars facts to be scraped
facts_url = 'https://space-facts.com/mars/'
tables = pd.read_html(facts_url)
tables

In [ ]:
mars_facts_df = tables[2]
mars_facts_df.columns = ["Description", "Value"]
mars_facts_df

In [ ]:
mars_html_table = mars_facts_df.to_html()
mars_html_table

In [ ]:
mars_html_table.replace('\n', '')

In [ ]:
print(mars_html_table)

In [ ]:
####################### Mars Hemispheres ##########################

In [ ]:
# Mars hemisphere name and image to be scraped
usgs_url = 'https://astrogeology.usgs.gov'
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(hemispheres_url)

hemispheres_html = browser.html

hemispheres_soup = BeautifulSoup(hemispheres_html, 'html.parser')

In [ ]:
# Mars hemispheres products data
all_mars_hemispheres = hemispheres_soup.find('div', class_='collapsible results')
mars_hemispheres = all_mars_hemispheres.find_all('div', class_='item')

hemisphere_image_urls = []

# Iterate through each hemisphere data
for i in mars_hemispheres:
    # Collect Title
    hemisphere = i.find('div', class_="description")
    title = hemisphere.h3.text
    
    # Collect image link by browsing to hemisphere page
    hemisphere_link = hemisphere.a["href"]    
    browser.visit(usgs_url + hemisphere_link)
    
    image_html = browser.html
    image_soup = BeautifulSoup(image_html, 'html.parser')
    
    image_link = image_soup.find('div', class_='downloads')
    image_url = image_link.find('li').a['href']

    # Create Dictionary to store title and url info
    image_dict = {}
    image_dict['title'] = title
    image_dict['img_url'] = image_url
    
    hemisphere_image_urls.append(image_dict)

print(hemisphere_image_urls)

In [ ]:

mars_dict = {
        "news_title": news_title,
        "news_p": news_p,
        "featured_image_url": featured_image_url,
        "mars_weather": mars_weather,
        "fact_table": str(mars_html_table),
        "hemisphere_images": hemisphere_image_urls
    }

In [ ]:
mars_dict